# Week3_2 Assignment

## [BASIC](#Basic) 
- 한국어 코퍼스를 로드해 **WordPiece Tokenzier를 학습**시킬 수 있다.
- 학습된 모델을 로드해 **encoding과 decoding을 수행**할 수 있다. 



### Reference
- [BertWordPieceTokenizer 학습 소개 한국어 블로그](https://monologg.kr/2020/04/27/wordpiece-vocab/)
- [huggingface python train tutorial](https://github.com/huggingface/tokenizers/blob/master/bindings/python/examples/train_bert_wordpiece.py)

In [1]:
import os 
import sys
import pandas as pd
import numpy as np

In [2]:
!pip install tokenizers

     |████████████████████████████████| 6.5 MB 9.2 MB/s 


In [3]:
import torch
from tokenizers import BertWordPieceTokenizer

In [4]:
# seed
seed = 7777
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [5]:
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla T4
cuda


## Basic

### 데이터 다운로드
- 내 구글 드라이브에 데이터를 다운 받은 후 코랩에 드라이브를 마운트하면 데이터를 영구적으로 사용할 수 있다. 
- [데이터 다운로드 출처](https://ratsgo.github.io/embedding/downloaddata.html)

In [6]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# 한국어 위키피디아 데이터 (토크나이즈되지 않은 텍스트) 로드
# !pip install gdown
# !gdown https://drive.google.com/u/0/uc?id=1kUecR7xO7bsHFmUI6AExtY5u2XXlObOG
# !unzip processed.zip
# 오류나서 직접 다운로드 후 업로드 

In [ ]:
%cd /content/drive/MyDrive/자연어/week3

/content/drive/MyDrive/자연어/week3


In [ ]:
_CUR_DIR = os.path.abspath(os.curdir)
print(f"My current directory : {_CUR_DIR}")
_DATA_DIR = os.path.join(_CUR_DIR, "processed")

My current directory : /content/drive/MyDrive/자연어/week3


### 한국어 위키피디아 코퍼스로 WordPiece tokenizer 학습
- 한국어 위키 

In [7]:
# processed_wiki_ko.txt 파일 불러오기
path = "/content/drive/MyDrive/자연어/week3/processed/processed_wiki_ko.txt"
docs = pd.read_csv(path, sep="\t", header=None)


In [8]:
print(f"# wiki documents: {len(docs):,}")

# wiki documents: 311,237


In [9]:
# Word Piece Tokenizer 인스턴스 생성
tokenizer = BertWordPieceTokenizer(
    clean_text=True, # 제어문자 제거 모든공백 기본공백으로 바꿀지 여부 
    handle_chinese_chars=True, # 공백을 둬서 중국어 문자를 처리할지 여부 
    strip_accents=False, # 악센트 제거할지 여부. True일경우 제거, 옵션이 지정되지 않은경우(None) 소문자로 값으로 결정. 악센트가 있는 character의 악센트를 제거하려면? (ex. é → e)
    lowercase=False, # True일 경우 대소문자 구분하지 않음. 한국어는 대소문자가 없는데 소문자 변환이 필요한지?
)

In [10]:
# train
# files = 'processed_wiki_ko.txt'
vocab_size = 30000
min_frequency = 2
special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]
limit_alphabet = 1000
wordpieces_prefix = '##'

tokenizer.train(
    files = path, 
    vocab_size = vocab_size, 
    min_frequency = min_frequency, # pair가 2회 이상 등장시 학습 
    show_progress = True, # 훈련중 진행과정 표시할지 여부
    special_tokens = special_tokens,
    limit_alphabet = limit_alphabet, # 병합전 초기 토큰 허용 개수 
    wordpieces_prefix = wordpieces_prefix 
)

In [11]:
tokenizer.save_model(".", "wordpiece")

['./wordpiece-vocab.txt']

### Encoding
- 저장된 토크나이즈 파일을 로드해 `BertWordPieceTokenizer` 인스턴스를 생성하고 다음을 수행하자. 
    - 사전(vocab)의 단어 개수를 출력
    - 문장을 토크나이징한 후 토큰 id와 토큰 string을 출력

In [12]:
save_path = '/content/drive/MyDrive/자연어/week3/wordpiece-vocab.txt'
tokenizer = BertWordPieceTokenizer(
    vocab = save_path,
    lowercase = False,
    strip_accents = False,
)

In [13]:
# 사전 단어 개수 출력
tokenizer.get_vocab_size()

30000

In [14]:
text = "안녕하세요. 버트를 사용한 모델입니다."

# 토크나이즈한 후 토큰의 id를 출력하라 
token_ids = tokenizer.encode(text).ids
print(token_ids)

# 토크나이즈한 후 각 토큰(string)을 출력하라.
tokens = tokenizer.encode(text).tokens
print(tokens)

[2, 7864, 20863, 16, 509, 3371, 5566, 2778, 5757, 16, 3]
['[CLS]', '안녕', '##하세요', '.', '버', '##트를', '사용한', '모델', '##입니다', '.', '[SEP]']


In [16]:
# 신조어를 토크나이징할 수 있는지 테스트해보자.
text = "어쩔티비"
tokens = tokenizer.encode(text).tokens
print(tokens)

['[CLS]', '[UNK]', '[SEP]']


In [17]:
text = "복세편살"
tokens = tokenizer.encode(text).tokens
print(tokens)

['[CLS]', '복', '##세', '##편', '##살', '[SEP]']


In [20]:
# 사전에 없는 단어는 어떻게 토크나이즈 되는가?
text = "조하진"
unknown_token_ids = tokenizer.encode(text).ids # 토큰 id
unknown_tokens = tokenizer.encode(text).tokens # 토큰
print(unknown_token_ids)
print(unknown_tokens)

[2, 751, 1182, 1218, 3]
['[CLS]', '조', '##하', '##진', '[SEP]']


### Decoding
- 토큰 id를 원래 문장으로 디코딩하자.

In [18]:
# 원래 문장: "안녕하세요. 버트를 사용한 모델입니다."
tokenizer.decode(token_ids)

'안녕하세요. 버트를 사용한 모델입니다.'

In [21]:
# 사전에 없는 단어는 어떻게 디코딩되는가?
tokenizer.decode(unknown_token_ids)

# wordpiece 임베딩을 사용해서 sub-word 단위로 단어를 나누므로 사전에 없는 단어여도 토큰화가 가능
# 어쩔티비 같은 경우 sub-word 분절이 안되서 unknown으로 나오는듯 ?

'조하진'